<a href="https://colab.research.google.com/github/gopiBNK/gopiBNK/blob/main/class_crud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st

class CRUDoperations:
    def __init__(self, db_manager):
        self.db_manager = db_manager

    def create_table(self):
        table_name = st.text_input("Enter Table Name")
        columns = st.text_input("Enter Columns in 'name:type' format (comma-separated, e.g., id:INTEGER, name:TEXT)")
        if table_name and columns:
            if st.button("Create Table"):
                try:
                    # Generate query for creating the table
                    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join([f'{col_name.strip()} {col_type.strip()}' for col_name, col_type in (col.split(':') for col in columns.split(','))])})"
                    self.db_manager.execute_query(query)
                    st.success(f"Table '{table_name}' created successfully!")
                except ValueError:
                    st.error("Error: Columns must be entered in 'name:type' format (e.g., id:INTEGER, name:TEXT).")

    def delete_table(self):
        table_name, _ = self.show_tables_and_columns()
        if table_name:
            # Extra confirmation step before deleting
            if st.button(f"Confirm Delete Table '{table_name}'"):
                self.db_manager.execute_query(f"DROP TABLE IF EXISTS {table_name}")
                st.success(f"Table '{table_name}' deleted successfully!")

    def alter_table(self):
        table_name, columns = self.show_tables_and_columns()
        if table_name:
            alter_option = st.selectbox("Choose Alter Option", ["Add Column", "Rename Table"])
            if alter_option == "Add Column":
                column_name = st.text_input("Enter New Column Name")
                column_type = st.text_input("Enter Column Type (e.g., TEXT, INTEGER, FLOAT)")
                if column_name and column_type:
                    if st.button("Add Column"):
                        self.db_manager.execute_query(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}")
                        st.success(f"Column '{column_name}' added successfully!")
            elif alter_option == "Rename Table":
                new_table_name = st.text_input("Enter New Table Name")
                if new_table_name:
                    if st.button("Rename Table"):
                        self.db_manager.execute_query(f"ALTER TABLE {table_name} RENAME TO {new_table_name}")
                        st.success(f"Table renamed to '{new_table_name}' successfully!")

    def update_table(self):
        table_name, columns = self.show_tables_and_columns()
        if table_name and columns is not None:
            set_column = st.selectbox("Select Column to Update", columns['name'])
            set_value = st.text_input("Enter New Value for the Column")
            condition = st.text_input("Enter Condition for Update (e.g., column=value)")
            if set_column and set_value and condition:
                try:
                    condition_col, condition_val = condition.split('=')
                    if st.button("Update Table"):
                        query = f"UPDATE {table_name} SET {set_column} = ? WHERE {condition_col.strip()} = ?"
                        self.db_manager.execute_query(query, (set_value, condition_val.strip()))
                        st.success(f"Updated column '{set_column}' in table '{table_name}' successfully!")
                except ValueError:
                    st.error("Condition must be in the format 'column=value'.")

    def view_tables(self):
        table_name, _ = self.show_tables_and_columns()
        if table_name:
            data = self.db_manager.fetch_data(f"SELECT * FROM {table_name}")
            st.write(data)

    def insert_data(self):
        table_name, columns = self.show_tables_and_columns()
        if table_name and columns is not None:
            column_names = [col for col in columns['name']]
            values = st.text_input(f"Enter Values for Columns: {', '.join(column_names)} (comma-separated)")
            if values:
                if st.button("Insert Data"):
                    try:
                        values_list = [value.strip() for value in values.split(',')]
                        if len(values_list) == len(column_names):
                            placeholders = ', '.join(['?' for _ in values_list])
                            query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({placeholders})"
                            self.db_manager.execute_query(query, values_list)
                            st.success(f"Data inserted into '{table_name}' successfully!")
                        else:
                            st.error("Error: The number of values must match the number of columns.")
                    except Exception as e:
                        st.error(f"Error inserting data: {e}")

    def show_tables_and_columns(self):
        """Fetch available tables and their columns."""
        tables = self.db_manager.fetch_data("SELECT name FROM sqlite_master WHERE type='table';")
        if tables is not None and not tables.empty:
            table_name = st.selectbox("Select a Table", tables['name'])
            columns = self.db_manager.fetch_data(f"PRAGMA table_info({table_name});")
            if columns is not None:
                st.write(f"Columns in '{table_name}':")
                st.write(columns[['name']])
                return table_name, columns
            else:
                st.warning("Unable to fetch columns for the selected table.")
        else:
            st.warning("No tables found.")
        return None, None
